In [1]:
!apt-get update
!apt-get install -y tesseract-ocr
!apt-get install -y libtesseract-dev
!pip install pytesseract Pillow

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,226 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,197 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [3,448 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://securi

In [7]:
!wget https://github.com/tesseract-ocr/tessdata/raw/main/tel.traineddata -P /usr/share/tesseract-ocr/4.00/tessdata/

--2025-01-02 15:47:21--  https://github.com/tesseract-ocr/tessdata/raw/main/tel.traineddata
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/tesseract-ocr/tessdata/main/tel.traineddata [following]
--2025-01-02 15:47:22--  https://raw.githubusercontent.com/tesseract-ocr/tessdata/main/tel.traineddata
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3315170 (3.2M) [application/octet-stream]
Saving to: ‘/usr/share/tesseract-ocr/4.00/tessdata/tel.traineddata’

tel.traineddata     100%[===================>]   3.16M  --.-KB/s    in 0.08s   

2025-01-02 15:47:22 (40.0 MB/s) - ‘/usr/share/tesseract-ocr/4.0

In [3]:
from PIL import Image, ImageOps, ImageFilter
import pytesseract

# Telugu language specification
lang = 'tel'
extracted_story = {}

def preprocess_image(image_path):
    """
    Preprocess the image for better OCR accuracy.
    - Converts to grayscale
    - Applies thresholding
    - Resizes the image for clarity
    """
    # Load the image
    image = Image.open(image_path)

    # Convert to grayscale
    #gray_image = ImageOps.grayscale(image)

    # Apply thresholding (binarization)
    #threshold_image = gray_image.point(lambda x: 0 if x < 128 else 255, '1')

    # Optional: Resize the image to make text clearer (increase width/height by 2x)
    #resized_image = threshold_image.resize(
    #    (threshold_image.width * 2, threshold_image.height * 2), Image.Resampling.LANCZOS
    #)

    # Optional: Apply a slight blur to reduce noise
    #final_image = resized_image.filter(ImageFilter.MedianFilter(size=3))

    #return final_image
    return image

def extract_text(image_path):
    """
    Extract Telugu text from an image using Tesseract OCR.
    """
    # Preprocess the image
    processed_image = preprocess_image(image_path)

    # Save the processed image for verification (optional)
    #processed_image.save('processed_image.png')

    # Perform OCR
    text = pytesseract.image_to_string(processed_image, lang=lang)

    return text


In [ ]:
#!rm -rf "/content/extracted_stories"

In [4]:
import zipfile
import os
import re

# Path to the zip file
zip_file_path = "/content/2-3-page-stories.zip"

# Extract the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Extract all files into a folder
    extract_path = "/content/extracted_stories"
    zip_ref.extractall(extract_path)

In [5]:
# Walk through the extracted files (including subfolders)

# Helper function to extract integer from file name
def extract_number(file_name):
    match = re.search(r'(\d+)$', os.path.splitext(file_name)[0])
    return int(match.group(1)) if match else float('inf')  # Default to inf if no number found

# Iterate through directories and files
for root, dirs, files in os.walk(extract_path):
  sorted_files = sorted(files, key=extract_number)
  story = []
  for file in sorted_files:
    image_path = os.path.join(root, file)
    story.append(extract_text(image_path))
  directory_name = os.path.basename(root)
  extracted_story[directory_name] = "".join(story)


In [6]:
import pandas as pd
df = pd.DataFrame(list(extracted_story.items()), columns=["title", "story"])

In [7]:
df

,title,story
0,extracted_stories,
1,story-31,పొౌరుగూళ్ళో జమీందారుగారి బంధువుల\nయింటి ఒక అమ్...
2,story-3,\n\nమళ్ళీ పౌర్ణమి వచ్చింది. వెన్సెట్లో తాతయ్య...
3,story-12,\n\nహిమాలయాలలో ఒకానొక చోట ఒక అడవి\n\nమనుపల జా...
4,story-24,\n\nరో\n\n| మహిమగ శ్రా.\n= క ఎడిబెద్ద\n\nహ...
5,story-15,\n\n \n\nవెళ్ళి ఆయనతో తన మనమడి. విషయం\nమొరబెట...
6,story-11,\n\n \n\nజౌర్ణమివెన్నట్లో తాతయ్య అప్పుడే భోజన...
7,story-2,\n\nసాధారణంగా గొప్పగొప్ప కవులందరూ\nఖభోజరాజును...
8,story-17,షావుకా రిచ్చిన పావలా జేబులో వేసుకుని\n\nమన్న -...
9,story-16,\n\n \n\n(బిటిమ గవర్నమెంటు కొత్తల్లో కెరటాల\...


In [8]:
df.to_csv("2-3-pages-stories.csv", index=False, encoding="utf-8")

In [4]:
from PIL import Image, ImageOps, ImageFilter
import pytesseract

In [9]:
lang = 'tel'
image = Image.open('/content/p-1.jpg')
text = pytesseract.image_to_string(image, lang=lang)
text

'"11౧. 1) 7 ర |\n\nఅతితెలిఏ\n\nఒక ఆసామి తన పొరుగు ఆసామివడ్డ ఒక కుండ అరువుతిసుకున్నాడు, ఆందులో వెన్న కాచుదామని\nఆతను నిప్పచేస్తూండగా ఎక్కడినుంచో పిల్లి వచ్చి కుండపైన కాళ్లుపెట్టి లేచి నిలబడింది, అదిచూసి\nఆతను పిల్లిని తరమబోయాడు. పిల్లి వెనక్కు దూకింది, కుండ అరుగుమీదినుంచి కిందవడి రెండే\nరెండు ముక్కలయింది.\n\nకుండ ఆరువు తెచ్చినవాడు. తడువుకోకుండా రెండుమాక్కలూ జిగురుపెట్టి అంటించి,\nహరుగువాడిపద్ధకు తీసుకుపోయి, “అదుగో నీ కుండ,” అని ఇచ్చేశాడు. పొరుగువాడు పగులు గమ\nనించ్మి * విమిటిది ?” అన్నాడు. ఆసామి, “నాకు తెలియదు,” అంటూ అంటికి వచ్చాడు.\n\nపౌరుగువాడు కోర్టులో “పిర్యాదు చేశాడు. ఆసామి ప్లీడరువద్దకు వెళ్లి నలహా అడిగాడు, ప్లీడరు\nఅంతా విని, “ జరిగినదానికి సామలెవరూ లేరుగనక నువు మూడువిథాలగా నసమర్టించుకోవచ్చు. నువు\nకుండ అరువు తీసుకున్నప్పుడే అది వగిలి ఉన్నదనపచ్చు; లేదా, నువు దానిని తెరిగి ఇచ్చాక పగిలిం\nదేమోననవచ్చు; అన్నిటికన్నా భేషయినదేమంటే, అనలు నువు కుండ తీసుకోలేదనవచ్చు,” అన్నాడు.\n\nకుండ ఖరీదు పావలే అయినా మంచి నలహా చెప్పినందుకు ప్లీకరుకు రూపాయి యిచ్చి\nఆసామి ఇంటికి వచ్చాడు.\n\nమర్చాడు కోర్టుల